In [2]:
from __future__ import print_function
import datetime
# import dash_table 
import pandas as pd
import numpy as np
import json
import itertools
import os
import sys


from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Layout, IntSlider
from ipywidgets.embed import embed_minimal_html

In [3]:
seriesNames_json_filename = '/Users/vir/Documents/GitHub/SeriesNames/seriesNames.json'
MRIData_json_filename = '/Users/vir/Documents/GitHub/MRIData/initialJsonADRCDataDump_copy.json'

In [4]:
#Read the json data
with open(MRIData_json_filename,'r') as fp:
    data=json.load(fp)



In [5]:
#Use for loop to remove the 'integer.' from each series name
seriescollection=[]
n_studies=0
for patient in data:
    for study in patient['studyList']:
        n_studies=n_studies+1 #Calculate the total number of studies
        seriescollection.append([series.split('.')[1].upper() for series in study['seriesList']])
        #append the modified string to the empty list

In [6]:
#flatten function in-place
flatten = lambda l: [item for sublist in l for item in sublist]
#flatten the seriescollection(list of list) to a single list
seriescollection_flat=flatten(seriescollection)
#make the series name unique
series_unique=np.array(list(set(flatten(seriescollection))))
#create a series name dictionary; keys:unique series name; value:frequency of the name
seriescollection_dict={i:seriescollection_flat.count(i) for i in series_unique}
series_unique = sorted(series_unique)

In [10]:
mapping_dict = {} #local dict for name mapping
common_name_table = [] #stores table of unique common names so far

current_json_dict = {} #stores the dict in the json file
current_series_names = [] #stores the actual series names that are currently stored
current_common_names = [] #stores the common names that are currently stored

temp_name = [] #used to store the actual name when user is pressing previous/next button
boolean_check = [False] #used to check if the user is pressing previous/next button
previous_button_counter = [-1] #used to check how many times the user has pressed previous/next button



if not os.path.isfile(seriesNames_json_filename): #creates json file if not already existing
    with open(seriesNames_json_filename, 'w') as outfile:  
        json.dump({}, outfile)

else:
    with open(seriesNames_json_filename, "rt") as infile:
        current_json_dict = json.load(infile)
    current_series_names = list(current_json_dict.keys())
    current_common_names = list(current_json_dict.values())
    
    
mapping_dict = current_json_dict.copy()
display_series_unique = list(set(series_unique) - set(current_series_names)) #stores the actual names that need to be displayed
display_series_unique = sorted(display_series_unique)
#stores the unique common names that need to be displayed
display_common_name_list = list(dict.fromkeys(current_common_names))
num_names = len(display_series_unique) #number of names left



for i in display_common_name_list: #creates unique common name table
    temp_list = []
    temp_list.append(i)
    common_name_table.append(temp_list)
    

#widgets creating for gui
out = widgets.Output(layout={'border': '1px solid black'})
series_name_iterator = iter(display_series_unique)
actual_name = widgets.Text(value=next(series_name_iterator),disabled=True)
common_name = widgets.Text(value='')
actual_name_label = widgets.Label(value='Actual Series Name:')
common_name_label = widgets.Label(value='Enter Common Name:')
table_label = widgets.Label(value='List of common names so far:')
names_remaining_label = widgets.Label(value='Number of names left: ')
names_remaining = widgets.Label(value=str(num_names) + ' of ' + str(len(series_unique)))

#submit button in tab1
submit_button1 = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Submit',
    icon='check',
    )

submit_button1.layout.width = '50%'

#back button on tab1
previous_button = widgets.Button(
    description='Back',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Back',
    icon='undo',
    )

previous_button.layout.width = '25%'
if len(mapping_dict)==0:
    previous_button.disabled = True
    
next_button = widgets.Button(
    description='Next',
    disabled=True,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Next',
    icon='arrow-right',
    )

next_button.layout.width = '25%'

table = widgets.HTML(
   '<table><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in common_name_table)
       )
)

change_name_label = widgets.Label(value='Enter Series Name to Change:')
new_name_label = widgets.Label(value='Enter New Common Name:')

change_name = widgets.Text(value='')
new_name = widgets.Text(value='')

submit_button2 = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Submit',
    icon='check',
    )

checkbox = widgets.Checkbox(
    value=False,
    description='Change all old common names to this new common name',
    disabled=False
)

checkbox.layout.width = '70%'

error_label = widgets.Label(value='')





with out:
    
    tab1_name_box = HBox([names_remaining_label,names_remaining])
    tab1_buttonbox = HBox([previous_button,next_button,submit_button1])
    tab1_vbox1 = VBox([tab1_name_box,actual_name_label,actual_name,common_name_label,common_name,tab1_buttonbox])
    tab1_vbox2 = VBox([table_label,table])
    tab1_hbox = HBox([tab1_vbox1,tab1_vbox2])
    tab1_hbox.layout.justify_content='space-around'
    
    tab2_hbox = HBox([new_name,checkbox])
    tab2_vbox = VBox([change_name_label,change_name,new_name_label,tab2_hbox,submit_button2,error_label])
    
    tab = widgets.Tab()
    tab.children = [tab1_hbox,tab2_vbox]
    tab.set_title(0,"Map New Names")
    tab.set_title(1,"Change Old Names")
    display(tab)
    
    

display(out)


def on_button1_clicked(b):
    
    common_name.value = common_name.value.upper()
    
    with open(seriesNames_json_filename, "rt") as f:
        json_dic1 = json.load(f)
    
    if boolean_check[0] == True: #if user is pressing previous/next button and then presses submit
        old_name = json_dic1[actual_name.value]
        count = 0
        for i in json_dic1:
            if json_dic1[i]==old_name:
                count = count + 1
        if count==1:
            display_common_name_list.remove(old_name)
            temp_list = []
            temp_list.append(old_name)
            common_name_table.remove(temp_list)
        
        for i in range(len(current_series_names)):
            if current_series_names[i] == actual_name.value:
                current_common_names[i] = common_name.value
                
    else: #adds new name mapping to relevant lists
        
        if actual_name.value not in current_series_names:
            current_series_names.append(actual_name.value)
            current_common_names.append(common_name.value)
    
    if common_name.value not in display_common_name_list:
        display_common_name_list.append(common_name.value)
        temp_list = []
        temp_list.append(common_name.value)
        common_name_table.append(temp_list)
    
    
    table.value = '<table><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in common_name_table)
       ) #fills table widget
    
    if actual_name.value not in mapping_dict:
            names_remaining.value = str(int((names_remaining.value).split()[0]) - 1) + ' of ' + str(len(series_unique))
        
    mapping_dict.update({actual_name.value:common_name.value})
    json_dic1.update({actual_name.value:common_name.value})

    with open(seriesNames_json_filename, "wt") as f:
        json.dump(json_dic1, f)
        
    if boolean_check[0] == True: #user picks up where they left of (current name)
        actual_name.value = temp_name[0]
        boolean_check.clear()
        boolean_check.append(False)
        
    else:
        
        try:
            actual_name.value = next(series_name_iterator)
            while actual_name.value not in display_series_unique:
                actual_name.value = next(series_name_iterator)
        except StopIteration: #no more names
            print('Done')
    
    if actual_name.value in mapping_dict:
        common_name.value = mapping_dict[actual_name.value]
    else:
        common_name.value = ''
        
    previous_button_counter.clear()
    previous_button_counter.append(-1) #resets counter and re - enables button
    previous_button.disabled = False
    next_button.disabled = True
    
    
    
    
def on_button2_clicked(b):
    
    new_name.value = new_name.value.upper()
    
    with open(seriesNames_json_filename, "rt") as f:
        json_dic2 = json.load(f)
        
    if change_name.value in json_dic2: #changing old name value
        
        old_name = json_dic2[change_name.value]
        
        if checkbox.value == False: #changing only one value 
        
            count = 0
            for i in json_dic2:
                if json_dic2[i]==old_name:
                    count = count + 1
            if count==1:
                display_common_name_list.remove(old_name)
                temp_list = []
                temp_list.append(old_name)
                common_name_table.remove(temp_list)
            
            for i in range(len(current_series_names)): #goes through dict to change one specific value
                if current_series_names[i] == change_name.value:
                    current_common_names[i] = new_name.value
                    
            mapping_dict.update({change_name.value:new_name.value})
            json_dic2.update({change_name.value:new_name.value}) 
                
                
        elif checkbox.value == True: #changing all values
            
            display_common_name_list.remove(old_name)
            temp_list = []
            temp_list.append(old_name)
            common_name_table.remove(temp_list)
            
            for i in range(len(current_common_names)):
                if current_common_names[i] == old_name:
                    current_common_names[i] = new_name.value
            
            mapping_dict.update({change_name.value:new_name.value})
            json_dic2.update({change_name.value:new_name.value})
            
            for i in json_dic2: #updates dicts with new values
                if json_dic2[i]==old_name:
                    mapping_dict.update({i:new_name.value})
                    json_dic2.update({i:new_name.value})
        
        if new_name.value not in display_common_name_list: #adds name to unique common name list
                display_common_name_list.append(new_name.value)
                temp_list = []
                temp_list.append(new_name.value)
                common_name_table.append(temp_list)
            
            
    elif change_name.value not in json_dic2: #new value being added
        
        if checkbox.value == True:
            error_label.value = str(change_name.value) + ' is not in the json file, therefore no old name for this entry exists, please untick the checkbox to proceed'
            return #user made a mistake
        
        if change_name.value in series_unique: #
            names_remaining.value = str(int((names_remaining.value).split()[0]) - 1) + ' of ' + str(len(series_unique))
            if change_name.value in display_series_unique:
                display_series_unique.remove(change_name.value)
        
        if new_name.value not in display_common_name_list:
                display_common_name_list.append(new_name.value)
                temp_list = []
                temp_list.append(new_name.value)
                common_name_table.append(temp_list)
        
        current_series_names.append(change_name.value)
        current_common_names.append(new_name.value)
        
        mapping_dict.update({change_name.value:new_name.value})
        json_dic2.update({change_name.value:new_name.value})
        
        if boolean_check[0]==True:
            local_counter = previous_button_counter[0]
            previous_button_counter.clear()
            previous_button_counter.append(local_counter-1)

    
    error_label.value = ''
    table.value = '<table><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in common_name_table)
       )
    
    with open(seriesNames_json_filename, "wt") as f:
        json.dump(json_dic2, f)
        
    
    if actual_name.value in mapping_dict:
        common_name.value = mapping_dict[actual_name.value]
    else:
        common_name.value = ''
        
    change_name.value = ''
    new_name.value = ''
    checkbox.value=False
        
        


def on_button3_clicked(b):
    
    with open(seriesNames_json_filename, "rt") as f:
        json_dic3 = json.load(f)
        
    local_previous_button_counter = previous_button_counter[0]
    
    if local_previous_button_counter == -1:
        temp_name.clear()
        temp_name.append(actual_name.value)
    
    while actual_name.value==current_series_names[local_previous_button_counter]:
        local_previous_button_counter = local_previous_button_counter - 1
        
    actual_name.value = current_series_names[local_previous_button_counter]
    common_name.value = current_common_names[local_previous_button_counter]

    
    previous_button_counter.clear()
    previous_button_counter.append(local_previous_button_counter - 1)
    
    boolean_check.clear()
    boolean_check.append(True)
    
    if previous_button_counter[0] <= -(len(mapping_dict)+1):
        previous_button.disabled = True
        
    next_button.disabled = False
    

def on_button4_clicked(b):
    
    local_next_button_counter = previous_button_counter[0] + 2
    if local_next_button_counter >= 0:
        actual_name.value = temp_name[0]
        if actual_name.value in mapping_dict:
            common_name.value = mapping_dict[actual_name.value]
        else:
            common_name.value = ''
        next_button.disabled = True
        boolean_check.clear()
        boolean_check.append(False)
    else:
        actual_name.value = current_series_names[local_next_button_counter]
        common_name.value = current_common_names[local_next_button_counter]
        
    if(actual_name.value==temp_name[0]):
        next_button.disabled = True
        boolean_check.clear()
        boolean_check.append(False)
        
    previous_button_counter.clear()
    previous_button_counter.append(local_next_button_counter-1)
    
# def changed(b):
#     print("Hello")
        
    

# slider = IntSlider(value=40)
# embed_minimal_html('export.html', views=[tab], title='Widgets export')
    
submit_button1.on_click(on_button1_clicked)
submit_button2.on_click(on_button2_clicked)
previous_button.on_click(on_button3_clicked)
next_button.on_click(on_button4_clicked)
# checkbox.observe(changed)



Output(layout=Layout(border='1px solid black'))

In [ ]:
ll = []

In [ ]:
# mapping_dict = {} #local dict for name mapping
# common_name_table = [] #stores table of unique common names so far

# current_json_dict = {} #stores the dict in the json file
# current_series_names = [] #stores the actual series names that are currently stored
# current_common_names = [] #stores the common names that are currently stored

# temp_name = [] #used to store the actual name when user is pressing previous/next button
# boolean_check = [False] #used to check if the user is pressing previous/next button
# previous_button_counter = [-1] #used to check how many times the user has pressed previous/next button



# if not os.path.isfile(seriesNames_json_filename): #creates json file if not already existing
#     with open(seriesNames_json_filename, 'w') as outfile:  
#         json.dump({}, outfile)

# else:
#     with open(seriesNames_json_filename, "rt") as infile:
#         current_json_dict = json.load(infile)
#     current_series_names = list(current_json_dict.keys())
#     current_common_names = list(current_json_dict.values())
    

# mapping_dict = current_json_dict.copy()
# display_series_unique = list(set(series_unique) - set(current_series_names)) #stores the actual names that need to be displayed
# display_common_name_list = list(set(current_common_names)) #stores the unique common names that need to be displayed
# num_names = len(display_series_unique) #number of names left



# for i in display_common_name_list: #creates unique common name table
#     temp_list = []
#     temp_list.append(i)
#     common_name_table.append(temp_list)
    

# #widgets creating for gui
# out = widgets.Output(layout={'border': '1px solid black'})
# series_name_iterator = iter(display_series_unique)
# actual_name = widgets.Text(value=next(series_name_iterator),disabled=True)
# common_name = widgets.Text(value='')
# actual_name_label = widgets.Label(value='Actual Series Name:')
# common_name_label = widgets.Label(value='Enter Common Name:')
# table_label = widgets.Label(value='List of common names so far:')
# names_remaining_label = widgets.Label(value='Number of names left: ')
# names_remaining = widgets.Label(value=str(num_names) + ' of ' + str(len(series_unique)))

# #submit button in tab1
# submit_button1 = widgets.Button(
#     description='Submit',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     tooltip='Submit',
#     icon='check',
#     )

# submit_button1.layout.width = '50%'

# #back button on tab1
# previous_button = widgets.Button(
#     description='Back',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     tooltip='Back',
#     icon='undo',
#     )

# previous_button.layout.width = '25%'
# if len(mapping_dict)==0:
#     previous_button.disabled = True
    
# next_button = widgets.Button(
#     description='Next',
#     disabled=True,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     tooltip='Next',
#     icon='undo',
#     )

# next_button.layout.width = '25%'

# table = widgets.HTML(
#    '<table><tr>{}</tr></table>'.format(
#        '</tr><tr>'.join(
#            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in common_name_table)
#        )
# )

# change_name_label = widgets.Label(value='Enter Series Name to Change:')
# new_name_label = widgets.Label(value='Enter New Common Name:')

# change_name = widgets.Text(value='')
# new_name = widgets.Text(value='')

# submit_button2 = widgets.Button(
#     description='Submit',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     tooltip='Submit',
#     icon='check',
#     )

# checkbox = widgets.Checkbox(
#     value=False,
#     description='Change all old common names to this new common name',
#     disabled=False
# )

# checkbox.layout.width = '70%'

# error_label = widgets.Label(value='')





# with out:
    
#     tab1_name_box = HBox([names_remaining_label,names_remaining])
#     tab1_buttonbox = HBox([previous_button,next_button,submit_button1])
#     tab1_vbox1 = VBox([tab1_name_box,actual_name_label,actual_name,common_name_label,common_name,tab1_buttonbox])
#     tab1_vbox2 = VBox([table_label,table])
#     tab1_hbox = HBox([tab1_vbox1,tab1_vbox2])
#     tab1_hbox.layout.justify_content='space-around'
    
#     tab2_hbox = HBox([new_name,checkbox])
#     tab2_vbox = VBox([change_name_label,change_name,new_name_label,tab2_hbox,submit_button2,error_label])
    
#     tab = widgets.Tab()
#     tab.children = [tab1_hbox,tab2_vbox]
#     tab.set_title(0,"Map New Names")
#     tab.set_title(1,"Change Old Names")
#     display(tab)
    
    

# display(out)


# def on_button1_clicked(b):
    
#     with open(seriesNames_json_filename, "rt") as f:
#         json_dic1 = json.load(f)
    
#     if boolean_check[0] == True: #if user is pressing previous/next button and then presses submit
#         old_name = json_dic1[actual_name.value]
#         count = 0
#         for i in json_dic1:
#             if json_dic1[i]==old_name:
#                 count = count + 1
#         if count==1:
#             display_common_name_list.remove(old_name)
#             temp_list = []
#             temp_list.append(old_name)
#             common_name_table.remove(temp_list)
        
#         for i in range(len(current_series_names)):
#             if current_series_names[i] == actual_name.value:
#                 current_common_names[i] = common_name.value
                
#     else: #adds new name mapping to relevant lists
        
#         if actual_name.value not in current_series_names:
#             current_series_names.append(actual_name.value)
#             current_common_names.append(common_name.value)
    
#     if common_name.value not in display_common_name_list:
#         display_common_name_list.append(common_name.value)
#         temp_list = []
#         temp_list.append(common_name.value)
#         common_name_table.append(temp_list)
    
    
#     table.value = '<table><tr>{}</tr></table>'.format(
#        '</tr><tr>'.join(
#            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in common_name_table)
#        ) #fills table widget
    
#     if actual_name.value not in mapping_dict:
#             names_remaining.value = str(int((names_remaining.value).split()[0]) - 1) + ' of ' + str(len(series_unique))
        
#     mapping_dict.update({actual_name.value:common_name.value})
#     json_dic1.update({actual_name.value:common_name.value})

#     with open(seriesNames_json_filename, "wt") as f:
#         json.dump(json_dic1, f)
        
#     if boolean_check[0] == True: #user picks up where they left of (current name)
#         actual_name.value = temp_name[0]
#         boolean_check.clear()
#         boolean_check.append(False)
        
#     else:
        
#         try:
#             actual_name.value = next(series_name_iterator)
#             while actual_name.value not in display_series_unique:
#                 actual_name.value = next(series_name_iterator)
#         except StopIteration: #no more names
#             print('Done')
    
#     if actual_name.value in mapping_dict:
#         common_name.value = mapping_dict[actual_name.value]
#     else:
#         common_name.value = ''
        
#     previous_button_counter.clear()
#     previous_button_counter.append(-1) #resets counter and re - enables button
#     previous_button.disabled = False
#     next_button.disabled = True
    
    
    
    
# def on_button2_clicked(b):
    
#     with open(seriesNames_json_filename, "rt") as f:
#         json_dic2 = json.load(f)
        
#     if change_name.value in json_dic2: #changing old name value
        
#         old_name = json_dic2[change_name.value]
        
#         if checkbox.value == False: #changing only one value 
        
#             count = 0
#             for i in json_dic2:
#                 if json_dic2[i]==old_name:
#                     count = count + 1
#             if count==1:
#                 display_common_name_list.remove(old_name)
#                 temp_list = []
#                 temp_list.append(old_name)
#                 common_name_table.remove(temp_list)
            
#             for i in range(len(current_series_names)): #goes through dict to change one specific value
#                 if current_series_names[i] == change_name.value:
#                     current_common_names[i] = new_name.value
                    
#             mapping_dict.update({change_name.value:new_name.value})
#             json_dic2.update({change_name.value:new_name.value}) 
                
                
#         elif checkbox.value == True: #changing all values
            
#             display_common_name_list.remove(old_name)
#             temp_list = []
#             temp_list.append(old_name)
#             common_name_table.remove(temp_list)
            
#             for i in range(len(current_common_names)):
#                 if current_common_names[i] == old_name.value:
#                     current_common_names[i] = new_name.value
            
#             mapping_dict.update({change_name.value:new_name.value})
#             json_dic2.update({change_name.value:new_name.value})
            
#             for i in json_dic2: #updates dicts with new values
#                 if json_dic2[i]==old_name:
#                     mapping_dict.update({i:new_name.value})
#                     json_dic2.update({i:new_name.value})
        
#         if new_name.value not in display_common_name_list: #adds name to unique common name list
#                 display_common_name_list.append(new_name.value)
#                 temp_list = []
#                 temp_list.append(new_name.value)
#                 common_name_table.append(temp_list)
            
            
#     elif change_name.value not in json_dic2: #new value being added
        
#         if checkbox.value == True:
#             error_label.value = str(change_name.value) + ' is not in the json file, therefore no old name for this entry exists, please untick the checkbox to proceed'
#             return #user made a mistake
        
#         if change_name.value in series_unique: #
#             names_remaining.value = str(int((names_remaining.value).split()[0]) - 1) + ' of ' + str(len(series_unique))
#             if change_name.value in display_series_unique:
#                 display_series_unique.remove(change_name.value)
        
#         if new_name.value not in display_common_name_list:
#                 display_common_name_list.append(new_name.value)
#                 temp_list = []
#                 temp_list.append(new_name.value)
#                 common_name_table.append(temp_list)
        
#         current_series_names.append(change_name.value)
#         current_common_names.append(new_name.value)
        
#         mapping_dict.update({change_name.value:new_name.value})
#         json_dic2.update({change_name.value:new_name.value})

    
#     error_label.value = ''
#     table.value = '<table><tr>{}</tr></table>'.format(
#        '</tr><tr>'.join(
#            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in common_name_table)
#        )
    
#     with open(seriesNames_json_filename, "wt") as f:
#         json.dump(json_dic2, f)
        
    
#     if actual_name.value in mapping_dict:
#         common_name.value = mapping_dict[actual_name.value]
#     else:
#         common_name.value = ''
        
#     change_name.value = ''
#     new_name.value = ''
#     checkbox.value=False
        
        


# def on_button3_clicked(b):
    
#     with open(seriesNames_json_filename, "rt") as f:
#         json_dic3 = json.load(f)
        
#     local_previous_button_counter = previous_button_counter[0]
    
#     if local_previous_button_counter == -1:
#         temp_name.clear()
#         temp_name.append(actual_name.value)
    
#     while actual_name.value==current_series_names[local_previous_button_counter]:
#         local_previous_button_counter = local_previous_button_counter - 1
        
#     actual_name.value = current_series_names[local_previous_button_counter]
#     common_name.value = current_common_names[local_previous_button_counter]

    
#     previous_button_counter.clear()
#     previous_button_counter.append(local_previous_button_counter - 1)
    
#     boolean_check.clear()
#     boolean_check.append(True)
    
#     if previous_button_counter[0] <= -(len(mapping_dict)+1):
#         previous_button.disabled = True
        
#     next_button.disabled = False
    

# def on_button4_clicked(b):
    
#     local_next_button_counter = previous_button_counter[0] + 2
#     if local_next_button_counter >= 0:
#         actual_name.value = temp_name[0]
#         if actual_name.value in mapping_dict:
#             common_name.value = mapping_dict[actual_name.value]
#         else:
#             common_name.value = ''
#         next_button.disabled = True
#         boolean_check.clear()
#         boolean_check.append(False)
#     else:
#         actual_name.value = current_series_names[local_next_button_counter]
#         common_name.value = current_common_names[local_next_button_counter]
        
#     if(actual_name.value==temp_name[0]):
#         next_button.disabled = True
#         boolean_check.clear()
#         boolean_check.append(False)
        
#     previous_button_counter.clear()
#     previous_button_counter.append(local_next_button_counter-1)
    
    
    
    
        
    
    
    
# submit_button1.on_click(on_button1_clicked)
# submit_button2.on_click(on_button2_clicked)
# previous_button.on_click(on_button3_clicked)
# next_button.on_click(on_button4_clicked)
